# 使用Apriori算法进行关联分析
## 基本概念
关联分析既不是监督学习也不是非监督学习，所以在python的机器学习库`sklearn`中没有API。准确的说，关联分析是一种统计分析概率和条件概率的方法，分别对应两个统计目标——发现频繁项集和发现关联规则。
## Apriori原理
假设我们一共有4个商品：商品0，商品1，商品2，商品3。

如下图所示，已知灰色部分{2,3}是非频繁项集，那么利用上面的知识，我们就可以知道{0,2,3}{1,2,3} {0,1,2,3}都是非频繁的。 也就是说，计算出{2,3}的支持度，知道它是非频繁的之后，就不需要再计算{0,2,3} {1,2,3} {0,1,2,3}的支持度，因为我们知道这些集合不会满足我们的要求。 使用该原理就可以避免项集数目的指数增长，从而在合理的时间内计算出频繁项集。
![infrequent](infrequent.png)

首先需要找到频繁项集，然后才能发现关联规则。

Apriori 算法是发现频繁项集的一种方法。 Apriori算法的两个输入参数分别是最小支持度和数据集。 该算法首先会生成所有单个物品的项集列表。 接着扫描交易记录来查看哪些项集满足最小支持度要求，那些不满足最小支持度要求的集合会被去掉。 然后对生下来的集合进行组合以生成包含两个元素的项集。 接下来再重新扫描交易记录，去掉不满足最小支持度的项集。 该过程重复进行直到所有项集被去掉。
### 生成候选项集
```
对数据集中的每一条交易记录transection
对每个候选项集candidate
    检查一下candidate是否是transection的子集:
        如果是则增加candidate的计数值
对每个候选项集
    如果其支持度不低于最小值，则保留该项集
    返回所有频繁项集列表
```

In [1]:
import numpy as np

#### 1. 我们首先创建一个数据集。数据集包含5中商品1、2、3、4、5，待会就通过Apriori算法发现频繁项集和关联规则。

In [2]:
def load_dataset():
    """
    创建一个数据集
    """
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
dataset = load_dataset()
dataset

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

#### 2. 通过生成的dataset初步生成只有一项的候选项集

In [4]:
def create_candidates1(dataset):
    candidates1 = []  # 只有一项的候选项集
    for transection in dataset:
        for item in transection:
            if [item] not in candidates1:
                candidates1.append([item])
    candidates1.sort()
    return list(map(frozenset, candidates1))

In [5]:
candidates1 = create_candidates1(dataset)
candidates1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

#### 3. 选择支持度大于最小支持度(min_support)的数据

In [6]:
def scan_dataset(dataset, candidates_k, min_support=0.5):
    candidate_cnt = {}  # 统计初步建立的候选项集中，项的出现次数
    for transection in dataset:
        for candidate in candidates_k:
            if candidate.issubset(transection):
                if candidate not in candidate_cnt:
                    candidate_cnt[candidate] = 1
                else:
                    candidate_cnt[candidate] += 1
    
    len_dataset = float(len(dataset))
    checked_candidates = []
    supports = {}
    for key in candidate_cnt:
        support = candidate_cnt[key] / len_dataset
        if support >= min_support:
            checked_candidates.append(key)
        supports[key] = support
    return checked_candidates, supports

In [7]:
dataset[0]

[1, 3, 4]

In [8]:
checked_candidates, supports = scan_dataset(dataset, candidates1)
checked_candidates

[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]

In [9]:
supports

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

上述候选集中，由于候选项集{4}的支持度只有0.25<阈值0.5，故被删除

### 组织完整的Apriori算法
Apriori算法伪代码：
```
当集合中项的个数大于0时：
    构建一个k个项组成的候选项集的列表
    检查数据已确认每个项集都是频繁的
    保留频繁项集并构建k+1项组成的候选项集的列表
```

In [10]:
def apriori_gen(checked_candidates_k, k):
    """
    生成不重复的下一级候选项集
    """
    next_candidates = []
    for i in range(len(checked_candidates_k) - 1):
        for j in range(i + 1, len(checked_candidates_k)):
            candidate1_pre = list(checked_candidates_k[i])[:k-2]
            candidate2_pre = list(checked_candidates_k[j])[:k-2]
            candidate1_pre.sort()
            candidate2_pre.sort()
            if candidate1_pre == candidate2_pre:
                next_candidates.append(checked_candidates_k[i] | checked_candidates_k[j])
    return next_candidates

In [11]:
def apriori(dataset, min_support=0.5):
    '''一个项的频繁项集'''
    candidates1 = create_candidates1(dataset)
    checked_candidates1, supports = scan_dataset(dataset, candidates1, min_support)
    
    '''迭代生成最终频繁项集'''
    rs_candidates = [checked_candidates1]
    k = 2
    while True:
        candidates_k = apriori_gen(rs_candidates[k-2], k)
#         print(candidates_k)
        checked_candidates_k, supports_k = scan_dataset(dataset, candidates_k, min_support)
        supports.update(supports_k)
        rs_candidates.append(checked_candidates_k)
        k += 1
        if not checked_candidates_k:
            break
    return rs_candidates, supports

In [12]:
rs_candidates, supports = apriori(dataset)
rs_candidates

[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({1, 3}), frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5})],
 [frozenset({2, 3, 5})],
 []]

In [13]:
supports

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({3, 5}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({1, 2}): 0.25,
 frozenset({1, 5}): 0.25,
 frozenset({2, 3, 5}): 0.5}

### 从频繁项集中挖掘关联规则
```
遍历项数大于1的频繁项集：
    如果频繁项集中项的个数大于2:
        按下图所示的进行迭代计算
    否则，频繁项集中项的个数等于2：
        计算条件概率
```
![apachecn_association_rule_demo_1](apachecn_association_rule_demo_1.jpg)

In [14]:
def generate_rules(rs_candidates, supports, min_conf=0.6):
    rules = []
    for i in range(1, len(rs_candidates)):
        for freq_set in rs_candidates[i]:
            freq_list = [frozenset([item]) for item in freq_set]
            if i > 1:
                rules_from_big_freq_set(freq_set, freq_list, supports, rules, min_conf)
            else:
                calc_conf(freq_set, freq_list, supports, rules, min_conf)
    return rules

In [21]:
def calc_conf(freq_set, freq_list, supports, rules, min_conf=0.6):
    checked_freq = []
    for related in freq_list:
        relating = freq_set - related
        conf = supports[freq_set] / supports[freq_set - related]
        if conf >= min_conf:
            print(relating, '-->', related, 'conf: ', conf)
            rules.append((relating, related, conf))
            checked_freq.append(related)
    return checked_freq

In [22]:
def rules_from_big_freq_set(freq_set, freq_list, supports, rules, min_conf=0.6):
    """
    freq_set: 项数相同的频繁项集的列表
    freq_list: freq_set中的项的列表，一开始是一个项的列表，随迭代增加
    """
    if len(freq_set) > (len(freq_list[0]) + 1):
        next_freq_list = apriori_gen(freq_list, len(freq_list[0]) + 1)
        next_freq_list = calc_conf(freq_set, next_freq_list, supports, rules, min_conf)
        if len(next_freq_list) > 1:
            rules_from_big_freq_set(freq_set, next_freq_list, supports, rules, min_conf)

In [23]:
rules = generate_rules(rs_candidates, supports)
rules

frozenset({3}) --> frozenset({1}) conf:  0.6666666666666666
frozenset({1}) --> frozenset({3}) conf:  1.0
frozenset({3}) --> frozenset({2}) conf:  0.6666666666666666
frozenset({2}) --> frozenset({3}) conf:  0.6666666666666666
frozenset({5}) --> frozenset({3}) conf:  0.6666666666666666
frozenset({3}) --> frozenset({5}) conf:  0.6666666666666666
frozenset({5}) --> frozenset({2}) conf:  1.0
frozenset({2}) --> frozenset({5}) conf:  1.0
frozenset({5}) --> frozenset({2, 3}) conf:  0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf:  0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf:  0.6666666666666666


[(frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]